<a href="https://colab.research.google.com/github/Gkulatheja/g_kulatheja/blob/master/bias_unbias_steriotype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import random

# Define sample text examples for each category
unbiased_samples = [
    "The weather is sunny today.",
    "The company achieved its quarterly goals.",
    "The book was published last month.",
    "The car is parked outside.",
    "The recipe requires 200 grams of flour."
]

bias_samples = [
    "The politician made a poor decision again.",
    "People from city X are always late.",
    "This brand is better than the rest.",
    "Only certain types of people are good at this.",
    "The new product is useless."
]

stereotype_samples = [
    "Men are always stronger than women.",
    "Women are bad at driving.",
    "People from country Y are lazy.",
    "Teenagers are irresponsible.",
    "Old people cannot use technology."
]

# Define labels for each category
labels = {"Unbiased": 0, "Bias": 1, "Stereotype": 2}

# Generate 100 samples per category (300 in total)
data = []
for _ in range(100):
    data.append({
        "text": random.choice(unbiased_samples),
        "label": labels["Unbiased"]
    })
    data.append({
        "text": random.choice(bias_samples),
        "label": labels["Bias"]
    })
    data.append({
        "text": random.choice(stereotype_samples),
        "label": labels["Stereotype"]
    })

# Shuffle the dataset
random.shuffle(data)

# Convert to DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv("/content/bias_stereotype_data.csv", index=False)

print("CSV file created with 300 samples for bias/stereotype detection.")

CSV file created with 300 samples for bias/stereotype detection.


In [2]:
# Required installations in Colab
!pip install transformers spacy scikit-learn pandas
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 53.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
# Imports
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset, random_split
from torch.nn import CrossEntropyLoss
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import spacy

# Load SpaCy model for linguistic features
nlp = spacy.load('en_core_web_sm')

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 labels: unbiased, bias, stereotype


ImportError: cannot import name 'AdamW' from 'transformers' (/usr/local/lib/python3.11/dist-packages/transformers/__init__.py)

In [ ]:
# Dataset class
class BiasStereotypeDataset(Dataset):
    def __init__(self, file_path, tokenizer):
        self.data = pd.read_csv(file_path)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]["text"]
        label = self.data.iloc[idx]["label"]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            truncation=True,
            return_token_type_ids=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs['token_type_ids'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


In [ ]:
# Specify file path for the dataset
file_path = '/content/bias_stereotype_data.csv'  # Adjust the path based on where your CSV file is uploaded
dataset = BiasStereotypeDataset(file_path, tokenizer)

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create dataloaders
batch_size = 16  # Adjust based on your needs
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)


In [ ]:
# Function to train model
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()
        # Move data to device
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'label'}
        labels = batch['label'].to(device)

        outputs = model(**inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)

# Evaluation function
def evaluate(model, dataloader, device):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'label'}
            outputs = model(**inputs)
            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(batch['label'].cpu().numpy())
    return accuracy_score(true_labels, predictions), classification_report(true_labels, predictions, target_names=['Unbiased', 'Bias', 'Stereotype'])


In [ ]:
# Import necessary libraries
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, random_split
from torch.nn import CrossEntropyLoss
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 labels: unbiased, bias, stereotype

# Load the dataset
class BiasStereotypeDataset(torch.utils.data.Dataset):
    def __init__(self, file_path, tokenizer):
        self.data = pd.read_csv(file_path)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]["text"]
        label = self.data.iloc[idx]["label"]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            truncation=True,
            return_token_type_ids=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs['token_type_ids'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Specify file path for the dataset
file_path = '/content/bias_stereotype_data.csv'  # Replace with your uploaded file path
dataset = BiasStereotypeDataset(file_path, tokenizer)

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create dataloaders
batch_size = 16  # Adjust batch size based on dataset size
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Set up optimizer, criterion
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = CrossEntropyLoss()

# Function to train model
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()
        # Move data to device
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'label'}
        labels = batch['label'].to(device)

        outputs = model(**inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)

# Evaluation function
def evaluate(model, dataloader, device):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'label'}
            outputs = model(**inputs)
            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(batch['label'].cpu().numpy())
    return accuracy_score(true_labels, predictions), classification_report(true_labels, predictions, target_names=['Unbiased', 'Bias', 'Stereotype'])

# Main training loop with early stopping
def train_and_evaluate(model, train_dataloader, val_dataloader, optimizer, criterion, epochs, patience):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    best_val_acc = 0
    patience_counter = 0

    for epoch in range(epochs):
        train_loss = train(model, train_dataloader, optimizer, criterion, device)
        print(f"Epoch {epoch + 1}/{epochs}")
        print(f"Train Loss: {train_loss:.4f}")

        # Clear cache to free memory
        torch.cuda.empty_cache()

        # Validation
        val_acc, val_report = evaluate(model, val_dataloader, device)
        print(f"Validation Accuracy: {val_acc:.4f}")
        print(val_report)

        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

# Define training parameters and start training
epochs = 2
patience = 2
train_and_evaluate(model, train_dataloader, val_dataloader, optimizer, criterion, epochs, patience)


In [ ]:
# Import necessary libraries
import torch

# Output Module for User Testing

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function for bias and stereotype prediction
def predict_bias_stereotype(model, tokenizer, text):
    inputs = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_token_type_ids=True,
        return_tensors='pt'
    )

    model.eval()
    with torch.no_grad():
        outputs = model(**{k: v.to(device) for k, v in inputs.items()})
        probs = torch.softmax(outputs.logits, dim=1)
        _, prediction = torch.max(probs, dim=1)
        confidence = probs[0][prediction.item()].item()

    label_map = {0: "Unbiased", 1: "Bias", 2: "Stereotype"}
    predicted_label = label_map[prediction.item()]

    return predicted_label, confidence

# User Testing Loop
def user_testing(model, tokenizer):
    print("Welcome to the Bias and Stereotype Prediction System!")
    print("Type 'exit' to stop the program.")

    while True:
        user_input = input("Enter a text sample for prediction: ")
        if user_input.lower() == 'exit':
            print("Exiting the program.")
            break

        predicted_label, confidence = predict_bias_stereotype(model, tokenizer, user_input)
        print(f"Predicted Label: {predicted_label} | Confidence: {confidence:.4f}")

# Call the user testing function
user_testing(model, tokenizer)
